In [130]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [131]:
def create_df_from_json(response, title):
    return pd.DataFrame(json_normalize(response[title]))

In [132]:
country = 'US'
currency = 'USD'
destinationplace = 'anywhere'
inboundpartialdate = 'anytime' # The return date. Format “yyyy-mm-dd”, “yyyy-mm” or “anytime”. Use empty string for oneway trip.
outboundpartialdate = 'anytime' # The outbound date. Format “yyyy-mm-dd”, “yyyy-mm” or “anytime”.
locale = 'en-US'
originplace = 'CMH'

url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/{country}/{currency}/{locale}/{originplace}/{destinationplace}/{outboundpartialdate}/{inboundpartialdate}"

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "2d32d04dd5msh5cb7c5e39103bebp1cd117jsn0f0dea8441dc"
    }

response = requests.request("GET", url, headers=headers).json()

In [133]:
places_df = create_df_from_json(response, 'Places')
places_df = places_df.drop(columns=['Type', 'SkyscannerCode', 'CityId'])

places_df.columns

<ipython-input-131-d69da532ae5c>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return pd.DataFrame(json_normalize(response[title]))


Index(['Name', 'PlaceId', 'IataCode', 'CityName', 'CountryName'], dtype='object')

In [134]:
carriers_dict = dict()

for item in response['Carriers']:
  carriers_dict[item['CarrierId']] = item['Name']

carriers_dict

{1154: 'Sky Airline',
 1618: 'Qatar Airways',
 1324: 'KLM',
 1190: 'transavia',
 952: 'Cathay Pacific',
 898: 'EVA Air',
 840: 'Air Algerie',
 421: 'Afriqiyah Airways',
 1467: 'Spirit Airlines',
 851: 'Alaska Airlines',
 1049: 'Etihad Airways',
 1044: 'Ethiopian Airlines',
 941: 'Copa',
 178: 'Divi Divi Air',
 1368: 'Lufthansa',
 1570: 'Philippine Airlines',
 1707: 'SAS',
 1218: 'Iberia',
 1710: 'Brussels Airlines',
 1680: 'Aerolineas Sosa',
 1523: 'Austrian Airlines',
 881: 'British Airways',
 1065: 'Frontier Airlines',
 1132: 'Air Greenland',
 1587: 'Zambian Airways',
 1760: 'TAP Air Portugal',
 838: 'Air France',
 1555: 'Pegasus Airlines',
 1793: 'United',
 870: 'jetBlue',
 1064: 'Flair Airlines',
 1541: 'Wingo',
 1530: 'Asiana Airlines',
 1361: 'LATAM Airlines',
 1860: 'Air Tahiti',
 1878: 'Wizz Air',
 1755: 'Turkish Airlines',
 835: 'Air Canada',
 1384: 'SWISS',
 852: 'Royal Air Maroc',
 954: 'China Southern',
 1276: 'Japan Airlines',
 857: 'Finnair',
 1082: 'Fiji Airways',
 2024:

In [135]:
quotes_df = create_df_from_json(response, 'Quotes')
quotes_df.head()

<ipython-input-131-d69da532ae5c>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return pd.DataFrame(json_normalize(response[title]))


,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate,InboundLeg.CarrierIds,InboundLeg.OriginId,InboundLeg.DestinationId,InboundLeg.DepartureDate
0,1,32,True,2021-09-25T12:51:00,[1065],45192,67759,2021-10-12T00:00:00,[1065],67759,45192,2021-10-26T00:00:00
1,2,46,False,2021-09-24T13:19:00,[1065],45192,67759,2021-10-19T00:00:00,[1467],67759,45192,2021-10-21T00:00:00
2,3,93,False,2021-09-24T13:21:00,[1467],45192,43086,2021-10-19T00:00:00,[1467],43086,45192,2021-11-02T00:00:00
3,4,111,False,2021-09-25T01:00:00,[1467],45192,82408,2021-11-08T00:00:00,[1467],82408,45192,2021-11-13T00:00:00
4,5,136,False,2021-09-24T13:24:00,[1467],45192,54785,2021-11-10T00:00:00,[1467],54785,45192,2021-12-04T00:00:00


In [136]:
nice_quotes_df = quotes_df.merge(places_df, left_on='OutboundLeg.OriginId', right_on='PlaceId', suffixes=('', '_OutboundOrigin'))
nice_quotes_df = nice_quotes_df.merge(places_df, left_on='OutboundLeg.DestinationId', right_on='PlaceId', suffixes=('', '_OutboundDestination'))
nice_quotes_df = nice_quotes_df.merge(places_df, left_on='InboundLeg.OriginId', right_on='PlaceId', suffixes=('', '_InboundOrigin'))
nice_quotes_df = nice_quotes_df.merge(places_df, left_on='InboundLeg.DestinationId', right_on='PlaceId', suffixes=('', '_InboundDestination'))

nice_quotes_df = nice_quotes_df.rename(columns={'Name':'Name_OutboundOrigin', 'Type':'Type_OutboundOrigin',
       'SkyscannerCode':'SkyscannerCode_OutboundOrigin', 'IataCode':'IataCode_OutboundOrigin', 'CityName':'CityName_OutboundOrigin', 
       'CityId':'CityId_OutboundOrigin', 'CountryName':'CountryName_OutboundOrigin'})

nice_quotes_df

,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate,InboundLeg.CarrierIds,InboundLeg.OriginId,...,Name_InboundOrigin,PlaceId_InboundOrigin,IataCode_InboundOrigin,CityName_InboundOrigin,CountryName_InboundOrigin,Name_InboundDestination,PlaceId_InboundDestination,IataCode_InboundDestination,CityName_InboundDestination,CountryName_InboundDestination
0,1,32,True,2021-09-25T12:51:00,[1065],45192,67759,2021-10-12T00:00:00,[1065],67759,...,Orlando International,67759,MCO,Orlando,United States,Port Columbus,45192,CMH,Columbus,United States
1,2,46,False,2021-09-24T13:19:00,[1065],45192,67759,2021-10-19T00:00:00,[1467],67759,...,Orlando International,67759,MCO,Orlando,United States,Port Columbus,45192,CMH,Columbus,United States
2,3,93,False,2021-09-24T13:21:00,[1467],45192,43086,2021-10-19T00:00:00,[1467],43086,...,Aguadilla,43086,BQN,Aguadilla,Puerto Rico,Port Columbus,45192,CMH,Columbus,United States
3,4,111,False,2021-09-25T01:00:00,[1467],45192,82408,2021-11-08T00:00:00,[1467],82408,...,Henry E. Rohlsen,82408,STX,St Croix Island,US Virgin Islands,Port Columbus,45192,CMH,Columbus,United States
4,5,136,False,2021-09-24T13:24:00,[1467],45192,54785,2021-11-10T00:00:00,[1467],54785,...,Guatemala City,54785,GUA,Guatemala City,Guatemala,Port Columbus,45192,CMH,Columbus,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,2196,False,2021-09-25T13:26:00,[1523],45192,54797,2021-10-26T00:00:00,[1523],54797,...,Guam A.B. Won Pat,54797,GUM,Guam,Guam,Port Columbus,45192,CMH,Columbus,United States
126,127,2242,False,2021-09-22T08:42:00,[1793],45192,74763,2021-10-23T00:00:00,[954],75064,...,Beijing Daxing,75064,PKX,Beijing,China,Port Columbus,45192,CMH,Columbus,United States
127,128,2382,False,2021-09-23T01:01:00,[1793],45192,54504,2021-10-24T00:00:00,[1132],54504,...,Nuuk,54504,GOH,Nuuk,Greenland,Port Columbus,45192,CMH,Columbus,United States
128,129,2413,False,2021-09-23T01:26:00,[1618],45192,40492,2021-10-24T00:00:00,[1618],40492,...,Auckland International,40492,AKL,Auckland,New Zealand,Port Columbus,45192,CMH,Columbus,United States


In [137]:
# Convert CarrierIds
def convert_carriers(id_list):
    carrier_list = []
    for id in id_list:
        carrier_list.append(carriers_dict[id])
    return ', '.join(carrier_list)

nice_quotes_df['OutboundLeg.Carriers'] = nice_quotes_df['OutboundLeg.CarrierIds'].apply(convert_carriers)
nice_quotes_df['InboundLeg.Carriers'] = nice_quotes_df['InboundLeg.CarrierIds'].apply(convert_carriers)

nice_quotes_df

,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate,InboundLeg.CarrierIds,InboundLeg.OriginId,...,IataCode_InboundOrigin,CityName_InboundOrigin,CountryName_InboundOrigin,Name_InboundDestination,PlaceId_InboundDestination,IataCode_InboundDestination,CityName_InboundDestination,CountryName_InboundDestination,OutboundLeg.Carriers,InboundLeg.Carriers
0,1,32,True,2021-09-25T12:51:00,[1065],45192,67759,2021-10-12T00:00:00,[1065],67759,...,MCO,Orlando,United States,Port Columbus,45192,CMH,Columbus,United States,Frontier Airlines,Frontier Airlines
1,2,46,False,2021-09-24T13:19:00,[1065],45192,67759,2021-10-19T00:00:00,[1467],67759,...,MCO,Orlando,United States,Port Columbus,45192,CMH,Columbus,United States,Frontier Airlines,Spirit Airlines
2,3,93,False,2021-09-24T13:21:00,[1467],45192,43086,2021-10-19T00:00:00,[1467],43086,...,BQN,Aguadilla,Puerto Rico,Port Columbus,45192,CMH,Columbus,United States,Spirit Airlines,Spirit Airlines
3,4,111,False,2021-09-25T01:00:00,[1467],45192,82408,2021-11-08T00:00:00,[1467],82408,...,STX,St Croix Island,US Virgin Islands,Port Columbus,45192,CMH,Columbus,United States,Spirit Airlines,Spirit Airlines
4,5,136,False,2021-09-24T13:24:00,[1467],45192,54785,2021-11-10T00:00:00,[1467],54785,...,GUA,Guatemala City,Guatemala,Port Columbus,45192,CMH,Columbus,United States,Spirit Airlines,Spirit Airlines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,2196,False,2021-09-25T13:26:00,[1523],45192,54797,2021-10-26T00:00:00,[1523],54797,...,GUM,Guam,Guam,Port Columbus,45192,CMH,Columbus,United States,Austrian Airlines,Austrian Airlines
126,127,2242,False,2021-09-22T08:42:00,[1793],45192,74763,2021-10-23T00:00:00,[954],75064,...,PKX,Beijing,China,Port Columbus,45192,CMH,Columbus,United States,United,China Southern
127,128,2382,False,2021-09-23T01:01:00,[1793],45192,54504,2021-10-24T00:00:00,[1132],54504,...,GOH,Nuuk,Greenland,Port Columbus,45192,CMH,Columbus,United States,United,Air Greenland
128,129,2413,False,2021-09-23T01:26:00,[1618],45192,40492,2021-10-24T00:00:00,[1618],40492,...,AKL,Auckland,New Zealand,Port Columbus,45192,CMH,Columbus,United States,Qatar Airways,Qatar Airways


In [138]:
nice_quotes_df.columns

Index(['QuoteId', 'MinPrice', 'Direct', 'QuoteDateTime',
       'OutboundLeg.CarrierIds', 'OutboundLeg.OriginId',
       'OutboundLeg.DestinationId', 'OutboundLeg.DepartureDate',
       'InboundLeg.CarrierIds', 'InboundLeg.OriginId',
       'InboundLeg.DestinationId', 'InboundLeg.DepartureDate',
       'Name_OutboundOrigin', 'PlaceId', 'IataCode_OutboundOrigin',
       'CityName_OutboundOrigin', 'CountryName_OutboundOrigin',
       'Name_OutboundDestination', 'PlaceId_OutboundDestination',
       'IataCode_OutboundDestination', 'CityName_OutboundDestination',
       'CountryName_OutboundDestination', 'Name_InboundOrigin',
       'PlaceId_InboundOrigin', 'IataCode_InboundOrigin',
       'CityName_InboundOrigin', 'CountryName_InboundOrigin',
       'Name_InboundDestination', 'PlaceId_InboundDestination',
       'IataCode_InboundDestination', 'CityName_InboundDestination',
       'CountryName_InboundDestination', 'OutboundLeg.Carriers',
       'InboundLeg.Carriers'],
      dtype='object')

In [142]:
latest_scan_df = nice_quotes_df[['QuoteId', 
'MinPrice', 
'Direct', 
'QuoteDateTime', 
'OutboundLeg.Carriers',
'InboundLeg.Carriers',
'OutboundLeg.DepartureDate',
'InboundLeg.DepartureDate', 
'Name_OutboundOrigin', 'IataCode_OutboundOrigin', 'CityName_OutboundOrigin', 'CountryName_OutboundOrigin',
'Name_OutboundDestination', 'IataCode_OutboundDestination', 'CityName_OutboundDestination', 'CountryName_OutboundDestination',
'IataCode_InboundOrigin', 
'IataCode_InboundDestination'
]]

In [145]:
latest_scan_df.loc[latest_scan_df['MinPrice'] < 100]

,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.Carriers,InboundLeg.Carriers,OutboundLeg.DepartureDate,InboundLeg.DepartureDate,Name_OutboundOrigin,IataCode_OutboundOrigin,CityName_OutboundOrigin,CountryName_OutboundOrigin,Name_OutboundDestination,IataCode_OutboundDestination,CityName_OutboundDestination,CountryName_OutboundDestination,IataCode_InboundOrigin,IataCode_InboundDestination
0,1,32,True,2021-09-25T12:51:00,Frontier Airlines,Frontier Airlines,2021-10-12T00:00:00,2021-10-26T00:00:00,Port Columbus,CMH,Columbus,United States,Orlando International,MCO,Orlando,United States,MCO,CMH
1,2,46,False,2021-09-24T13:19:00,Frontier Airlines,Spirit Airlines,2021-10-19T00:00:00,2021-10-21T00:00:00,Port Columbus,CMH,Columbus,United States,Orlando International,MCO,Orlando,United States,MCO,CMH
2,3,93,False,2021-09-24T13:21:00,Spirit Airlines,Spirit Airlines,2021-10-19T00:00:00,2021-11-02T00:00:00,Port Columbus,CMH,Columbus,United States,Aguadilla,BQN,Aguadilla,Puerto Rico,BQN,CMH
